In [3]:
import numpy as np
import pandas as pd
import os
import sys
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models
import torch.optim as optim
import torch.utils.data as data_utils
from torch.utils.data import DataLoader, Dataset
import cv2
from tqdm import tqdm
from sklearn.model_selection import KFold

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dir = r"C:\dacon\\data\\"
train_dir = dir + "dirty_mnist_2nd\\"
train_label_dir = pd.read_csv(r"C:\dacon\\data\\dirty_mnist_2nd_answer.csv")

class ToTensor():
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = image.transpose((2, 0, 1))
        return {'image':torch.FloatTensor(image), 'label':torch.FloatTensor(label)}

to_tensor = T.Compose([
    # T.Resize((224, 224)),
    ToTensor()
])

class Generator(Dataset):
    def __init__(self, train_dir, train_label_dir, transforms = to_tensor, augmentations=None):
        self.x_data = train_dir
        self.x_label = train_label_dir
        self.transforms = transforms
        self.augmentations = augmentations
            
    def __len__(self):
        return len(self.x_label)
    
    def __getitem__(self, idx):
        train = cv2.imread(self.train_dir+str(self.x_label.iloc[idx,0]).zfill(5)+'.png', cv2.IMREAD_COLOR)
        train = cv2.resize(train,(299, 299) )
        train = (train/255).astype('float32')
        label = self.x_label.iloc[idx, 1:].values.astype('float32')
        sample = {'image':image, 'label':label}
        if self.transforms is not None:
            sample = self.transforms(sample)
        return sample
    
    
    
        
        
        
        

In [15]:
kfold = KFold(n_splits=5, shuffle=False, random_state=0)
epoch_size = 5
batch_size = 8
    
best_model = []
for fold_index, (train_idx, valid_idx) in enumerate(kfold.split(train_label_dir), 1):
        
        print(f'[fold:] {fold_index}')
        torch.cuda.empty_cache()#gpu에서 메모리 내려놓음
        
        train_label = train_label_dir.iloc[train_idx]
        valid_label = train_label_dir.iloc[valid_idx]
        train_image = Generator(train_dir, train_label)
        valid_image = Generator(train_dir, valid_label)
        
        train_loader = DataLoader(
            train_image,
            batch_size=batch_size,
            shuffle = False,
            num_workers=0                  
        )
        valid_loader = DataLoader(
            valid_image,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0                  
        )

[fold:] 1
[fold:] 2
[fold:] 3
[fold:] 4
[fold:] 5


In [20]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()

        # bias=Fasle, because BN after conv includes bias.
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, bias=False, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.conv(x)
        return x

In [21]:
#STEM
class Stem(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            BasicConv2d(3, 32, 3, stride=2, padding=0), # 149 x 149 x 32
            BasicConv2d(32, 32, 3, stride=1, padding=0), # 147 x 147 x 32
            BasicConv2d(32, 64, 3, stride=1, padding=1), # 147 x 147 x 64 
        )

        self.branch3x3_conv = BasicConv2d(64, 96, 3, stride=2, padding=0) # 73x73x96

        #  kernel_size=4: 피쳐맵 크기 73, kernel_size=3: 피쳐맵 크기 74
        self.branch3x3_pool = nn.MaxPool2d(4, stride=2, padding=1) # 73x73x64

        self.branch7x7a = nn.Sequential(
            BasicConv2d(160, 64, 1, stride=1, padding=0),
            BasicConv2d(64, 96, 3, stride=1, padding=0)
        ) # 71x71x96

        self.branch7x7b = nn.Sequential(
            BasicConv2d(160, 64, 1, stride=1, padding=0),
            BasicConv2d(64, 64, (7,1), stride=1, padding=(3,0)),
            BasicConv2d(64, 64, (1,7), stride=1, padding=(0,3)),
            BasicConv2d(64, 96, 3, stride=1, padding=0)
        ) # 71x71x96

        self.branchpoola = BasicConv2d(192, 192, 3, stride=2, padding=0) # 35x35x192

        #  kernel_size=4: 피쳐맵 크기 73, kernel_size=3: 피쳐맵 크기 74
        self.branchpoolb = nn.MaxPool2d(4, 2, 1) # 35x35x192


    def forward(self, x):
        x = self.conv1(x)
        x = torch.cat((self.branch3x3_conv(x), self.branch3x3_pool(x)), dim=1)
        x = torch.cat((self.branch7x7a(x), self.branch7x7b(x)), dim=1)
        x = torch.cat((self.branchpoola(x), self.branchpoolb(x)), dim=1)
        return x

In [22]:
# check Stem
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.randn((3, 3, 299, 299)).to(device)
model = Stem().to(device)
output_Stem = model(x)
print('Input size:', x.size())
print('Stem output size:', output_Stem.size())

Input size: torch.Size([3, 3, 299, 299])
Stem output size: torch.Size([3, 384, 35, 35])


In [ ]:
class Inception_Resnet_A(nn.Module):
    def __init__(self, in_channels):
        super().__init__()

        self.branch1x1 = BasicConv2d(in_channels, 32, 1, stride=1, padding=0)

        self.branch3x3 = nn.Sequential(
            BasicConv2d(in_channels, 32, 1, stride=1, padding=0),
            BasicConv2d(32, 32, 3, stride=1, padding=1)
        )

        self.branch3x3stack = nn.Sequential(
            BasicConv2d(in_channels, 32, 1, stride=1, padding=0),
            BasicConv2d(32, 32, 3, stride=1, padding=1),
            BasicConv2d(48, 64, 3, stride=1, padding=1)
        )
        
        self.reduction1x1 = nn.Conv2d(128, 384, 1, stride=1, padding=0)
        self.shortcut = nn.Conv2d(in_channels, 384, 1, stride=1, padding=0)
        self.bn = nn.BatchNorm2d(384)
        self.relu = nn.ReLU()

    def forward(self, x):
        x_shortcut = self.shortcut(x)
        x = torch.cat((self.branch1x1(x), self.branch3x3(x), self.branch3x3stack(x)), dim=1)
        x = self.reduction1x1(x)
        x = self.bn(x_shortcut + x)
        x = self.relu(x)
        return x